# Improve C# Code Performance 

- [reference](https://medium.com/stackademic/stop-writing-slow-c-span-and-memory-are-the-upgrade-your-codebase-desperately-needs-d6d61240682b). 

In [1]:
using System;

var input = "123,45.67";
var parts = input.Split(','); //Split allocates memory for a new array, Each part becomes a new string in memory
//modern dotnet use splice with span to avoid allocations
var id = int.Parse(parts[0]);
var score = double.Parse(parts[1]);

Console.WriteLine($"ID: {id}, Score: {score}");

ID: 123, Score: 45.67


- Span<T> is a stack-only type that represents a contiguous region of arbitrary memory.  
- It can be used to work with slices of arrays, strings, or unmanaged memory without additional allocations.
- Using Span<T> can help improve performance by reducing memory allocations and garbage collection overhead, especially in high-performance scenarios.


In [2]:
{

    ReadOnlySpan<char> span = input.AsSpan(); // var 키워드 사용으로 컴파일러가 타입 추론
    var prefix = span.Split(",");
    foreach(var item in prefix)
    {
        Console.WriteLine(span[item].ToString());
    }
}

123
45.67


In [ ]:
//Span Enumerator Example

using System;
using System.Threading.Tasks;

public class Program
{
    private static readonly byte[] _array = new byte[5];

    public static void Main()
    {
        new Random(42).NextBytes(_array);
        Span<byte> span = _array;

        Task.Run( () => ClearContents() );

       EnumerateSpan(span);
    }

    public static void ClearContents()
    {
        Task.Delay(20).Wait();
        lock (_array)
        {
           Array.Clear(_array, 0, _array.Length);
        }
    }

    public static void EnumerateSpan(Span<byte> span)
    {
        lock (_array)
        {
            foreach (byte element in span) //Span<T>.Enumerator is a ref struct.
            {
                Console.WriteLine(element);
                Task.Delay(10).Wait();
            }
        }

    }
}
// The example displays output like the following:
//     62
//     23
//     186
//     0
//     0
Program.Main();

62
23
186
150
174


# ReadOnlySpan<T> 

- it cannot mutate the underlying data. It is perfect for reading strings, byte arrays, network buffers, and data you did not allocate.

# Span<T> 

- it's the mutable version. You use it when you want to write into memory. For example, formatting content into a buffer.

In [3]:
{
    var value = 12345;
    Span<char> buffer = stackalloc char[32];
    bool ok = value.TryFormat(buffer, out int written);
}

In [ ]:
//wrong buffer
{
    var buffer = new char[32]; //heap allocation
}

//No heap usage. No GC involvement. No object allocation.
{
    Span<char> buffer = stackalloc char[32];
}


In [4]:
using System;

public static class StackAllocExample
{
    public static void Main()
    {
        string original = "Hello, StackAlloc!";
        Console.WriteLine($"Original: {original}");
        
        string reversed = ReverseString(original);
        Console.WriteLine($"Reversed: {reversed}");
    }

    public static string ReverseString(string input)
    {
        if (string.IsNullOrEmpty(input)) return input;

        // 1. 힙(Heap)이 아닌 스택(Stack)에 메모리를 할당합니다.
        // new char[input.Length]를 쓰면 GC가 나중에 수거해야 하지만,
        // stackalloc은 메서드가 끝나면 즉시 자동 소멸됩니다.
        Span<char> buffer = stackalloc char[input.Length];

        // 2. 데이터를 뒤집어서 스택 버퍼에 저장합니다.
        for (int i = 0; i < input.Length; i++)
        {
            buffer[i] = input[input.Length - 1 - i];
        }

        // 3. 최종 결과만 문자열(힙)로 변환하여 반환합니다.
        // 중간 연산 과정에서는 힙 할당이 전혀 발생하지 않았습니다.
        return buffer.ToString();
    }
}
StackAllocExample.Main();

Original: Hello, StackAlloc!
Reversed: !collAkcatS ,olleH


# stackalloc을 활용한 고성능 메모리 할당 예제
`stackalloc`은 힙(Heap)이 아닌 **스택(Stack)** 메모리에 배열을 할당하는 키워드입니다. 이를 `Span<T>`과 함께 사용하면 가비지 컬렉터(GC)의 관리 없이 매우 빠르게 임시 버퍼를 생성하고 사용할 수 있습니다. 주로 짧은 시간 동안 유지되는 데이터 처리에 사용됩니다.

## 예시 파일
[Microsoft 공식 stackalloc 예제 코드](https://github.com/dotnet/samples/blob/main/snippets/csharp/language-reference/operators/Stackalloc/Program.cs)

`stackalloc`을 사용하면 `new` 키워드를 사용할 때 발생하는 힙 메모리 할당과 나중에 GC가 이를 청소하는 비용을 절약할 수 있습니다.

아래는 문자열을 뒤집는(Reverse) 기능을 구현할 때, 중간 임시 저장소를 힙 배열(`char[]`) 대신 `stackalloc`을 사용하여 최적화한 예제입니다.

### 핵심 포인트
1.  **속도 향상**: 스택 할당은 CPU 명령 몇 개로 끝나므로 힙 할당보다 훨씬 빠릅니다.
2.  **GC 부담 없음**: 메서드(`ReverseString`)가 종료되면 스택 메모리는 자동으로 정리되므로 GC가 관여하지 않습니다.
3.  **Span과의 결합**: 과거에는 `unsafe` 키워드와 포인터를 써야 했지만, 현재는 `Span<T>` 덕분에 안전하게(Safe context) 사용할 수 있습니다.
4.  **주의사항**: 스택 메모리는 크기가 작습니다(보통 1MB). 너무 큰 배열(예: 수만 개 이상의 요소)을 `stackalloc`으로 할당하면 **StackOverflowException**이 발생하여 프로그램이 종료될 수 있습니다. 따라서 작은 크기의 버퍼에만 사용해야 합니다.

### 추가 자료
- [stackalloc 식(C# 참조)](https://learn.microsoft.com/ko-kr/dotnet/csharp/language-reference/operators/stackalloc)
- [Span<T> 및 메모리 효율성 가이드](https://learn.microsoft.com/ko-kr/dotnet/standard/memory-and-spans/memory-and-spans-usage-guidelines)

In [ ]:
using System;
using System.IO;
using System.Threading.Tasks;

public class Processor
{
    private Memory<byte> _buffer;

    public Processor(int size)
    {
        _buffer = new byte[size];
    }

    public async Task FillAsync(Stream source)
    {
        int read = await source.ReadAsync(_buffer);
        Span<byte> span = _buffer.Span.Slice(0, read);
        Process(span);
    }
    private void Process(Span<byte> data)
    {
        Console.WriteLine($"Processing {data.Length} bytes...");

        long sum = 0;
        
        // Span을 순회하며 데이터 처리 (배열처럼 인덱스 접근 가능)
        foreach (byte b in data)
        {
            sum += b;
        }

        Console.WriteLine($"Data Sum: {sum}");

        // 예시: 첫 번째 바이트 값 확인
        if (data.Length > 0)
        {
            Console.WriteLine($"First Byte: {data[0]}");
        }
    }
}

## 핵심 설명
- Span<byte> 매개변수: Process 메서드는 Memory<byte>가 아닌 Span<byte>를 받습니다. 이는 _buffer.Span 프로퍼티를 통해 매우 저렴하게 생성됩니다.
- 동기 메서드: Span<T>는 ref struct이기 때문에 힙에 저장될 수 없습니다. 따라서 async/await 메서드의 매개변수나 지역 변수로 사용할 수 없어, Process는 반드시 동기 메서드(void 등)여야 합니다.
- 효율성: Slice(0, read)를 호출해도 새로운 배열이 생성되지 않고, 기존 메모리의 특정 영역을 가리키는 뷰(View)만 생성되므로 성능이 매우 뛰어납니다.

# Memory<T>와 Span<T>의 차이 및 사용 패턴
`Span<T>`은 **스택(Stack)**에만 존재할 수 있어 빠르지만 제약이 많은 반면, `Memory<T>`는 **힙(Heap)**에 저장될 수 있어 비동기 작업이나 클래스 필드로 사용할 수 있습니다. `Memory<T>`로 데이터를 저장/전달하고, 실제 데이터를 처리할 때는 `Span`으로 변환하여 고성능으로 작업하는 것이 표준 패턴입니다.

## 예시 파일
[Memory<T>와 Span<T> 사용 가이드 (Microsoft Docs)](https://learn.microsoft.com/ko-kr/dotnet/standard/memory-and-spans/memory-and-spans-usage-guidelines)

## 답변

### 1. 차이점 핵심 요약

| 특성 | Span&lt;T&gt; | Memory&lt;T&gt; |
| :--- | :--- | :--- |
| **메모리 위치** | **스택(Stack) 전용** (`ref struct`) | 스택 또는 **힙(Heap)** 모두 가능 |
| **클래스 필드** | 불가능 (힙에 저장 불가) | **가능** (클래스 멤버로 저장 가능) |
| **비동기(async)** | `await` 전후로 유지 불가 | **가능** (`await` 넘어서 유지 가능) |
| **용도** | 데이터를 **처리(Processing)**할 때 | 데이터를 **저장(Storage)**하거나 전달할 때 |
| **성능** | 매우 빠름 (인덱싱 최적화) | Span보다 약간 느림 (변환 필요) |

### 2. 왜 Memory 선언 후 Span을 호출하는가?

`Memory<T>`를 선언해두고 굳이 `.Span` 프로퍼티를 호출해서 `Span<T>`으로 변환해 사용하는 이유는 **"역할의 분리"**와 **"성능"** 때문입니다.

1.  **저장과 이동 (Memory의 역할)**:
    *   비동기 메서드(`async/await`)나 클래스의 필드(`private Memory<byte> _buffer`)처럼 데이터가 힙에 오랫동안 살아있어야 하는 경우 `Memory<T>`에 담아둡니다. `Span`은 여기에 존재할 수 없기 때문입니다.

2.  **고속 처리 (Span의 역할)**:
    *   실제로 데이터를 읽거나 쓸 때(Loop 돌기, 파싱 등)는 `Span`이 훨씬 빠릅니다.
    *   `Memory<T>` 자체의 인덱서보다 `Span`으로 변환한 뒤 접근하는 것이 CPU 차원에서 최적화되어 있습니다.
    *   따라서 **"저장은 Memory에, 처리는 Span으로"** 하는 패턴을 사용합니다.

### 3. 코드 예시

```csharp
using System;
using System.Threading.Tasks;

public class AsyncDataHandler
{
    // 1. 클래스 필드에는 Span을 쓸 수 없으므로 Memory<byte>를 사용해 저장합니다.
    private Memory<byte> _storage;

    public AsyncDataHandler(int size)
    {
        _storage = new byte[size];
    }

    // 2. 비동기 메서드에서는 Memory<T>를 매개변수로 받거나 사용합니다.
    public async Task LoadAndProcessAsync()
    {
        // 비동기 작업: 여기서는 Memory<T> 상태로 전달해야 합니다.
        // (Span은 await를 건너뛸 수 없음)
        await SimulateNetworkReadAsync(_storage);

        // 3. 데이터 처리가 시작되는 시점에 .Span을 호출합니다.
        // 이 시점부터는 동기적으로 동작하며, Span의 고성능 이점을 활용합니다.
        Span<byte> spanView = _storage.Span;
        
        // 실제 데이터 조작은 Span으로 수행
        for (int i = 0; i < spanView.Length; i++)
        {
            spanView[i] = (byte)(spanView[i] * 2);
        }
    }

    private async Task SimulateNetworkReadAsync(Memory<byte> buffer)
    {
        await Task.Delay(100); // IO 시뮬레이션
        // ... 데이터 채우기 ...
    }
}
```

### 요약
- **Memory<T>**: 데이터를 **가방**에 넣어두는 것과 같습니다. 가방은 선반(Heap)에 올릴 수도 있고 친구에게 택배(Async)로 보낼 수도 있습니다.
- **Span<T>**: 가방에서 물건을 꺼내 **손**에 쥐고 작업하는 것과 같습니다. 손에 쥐고 있어야(Stack) 가장 빠르게 작업할 수 있지만, 손에 쥔 채로 잠을 자거나(await) 멀리 이동할 수는 없습니다.
- **패턴**: 가방(`Memory`)에 담아 이동하다가, 작업할 때만 손(`Span`)에 쥐고 처리합니다.

### 추가 자료
- [Memory&lt;T&gt; 및 Span&lt;T&gt; 사용 지침](https://learn.microsoft.com/ko-kr/dotnet/standard/memory-and-spans/memory-and-spans-usage-guidelines)
- [C#의 모든 Span에 대한 설명 (Microsoft Blog)](https://devblogs.microsoft.com/dotnet/all-about-span-part-1-introduction/)